#0.Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Sep 17 13:10:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    23W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 1.Dataset

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [ ]:
import json
from datasets import Dataset, DatasetDict

with open('/content/drive/MyDrive/절미네 가족들/1.jiwon/1.data/5.QA_data(json)/QA_data.json', 'r') as f:
    data_before1 = json.load(f)

In [ ]:
data_before1_fixed = []

for item in data_before1:
    fixed_item = {
        "instruction": str(item["instruction"]),
        "input": str(item["input"]),
        "output": str(item["output"])
    }
    data_before1_fixed.append(fixed_item)

data_as_dict = {
    "instruction": [item["instruction"] for item in data_before1_fixed],
    "input": [item["input"] for item in data_before1_fixed],
    "output": [item["output"] for item in data_before1_fixed],
}

In [ ]:
data_before2 = Dataset.from_dict(data_as_dict)

# 'train'이라는 split에 위에서 생성한 Dataset을 할당하여 DatasetDict를 생성
data = DatasetDict({"train": data_before2})

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 44438
    })
})

In [ ]:
# data
data = data.map(
    lambda x:
    {'text': f"### 명령어: {x['instruction']}\n\n###맥락: {x['input']}\n\n### 답변: {x['output']}<|endoftext|>" }
    if x['input'] else
    {'text':f"### 명령어: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"},
)
# data
# data = data.map(
#     lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
# )

Map:   0%|          | 0/44438 [00:00<?, ? examples/s]

# 2.Modeling

## 베이스 모델 호출 및 데이터셋 가공

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "nlpai-lab/kullm-polyglot-5.8b-v2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

In [ ]:
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/44438 [00:00<?, ? examples/s]

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 3670016 || all params: 3070156800 || trainable%: 0.11953838970048696


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [ ]:
!nvidia-smi

Sun Sep 17 13:14:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    40W / 300W |   5262MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Train

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="./outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,2.463100
20,2.262600
30,2.217900
40,2.223200
50,2.244800
60,2.280800
70,2.219000
80,2.122100
90,2.135300
100,2.287200


TrainOutput(global_step=500, training_loss=2.1622845306396483, metrics={'train_runtime': 488.9685, 'train_samples_per_second': 2.045, 'train_steps_per_second': 1.023, 'total_flos': 1.6983219666518016e+16, 'train_loss': 2.1622845306396483, 'epoch': 0.02})

## 학습파일 내보내기

In [ ]:
output_dir="./outputs"

In [ ]:
trainer.model.save_pretrained(output_dir)

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)

In [ ]:
# import os
# import shutil

# # 디렉토리 생성
# dest_dir = '/content/drive/MyDrive/4bit_kullm_5.8b_version2'
# if not os.path.exists(dest_dir):
#     os.mkdir(dest_dir)

# # 파일 복사
# src_dir = 'outputs'
# shutil.copytree(src_dir, os.path.join(dest_dir, 'outputs'))

'/content/drive/MyDrive/4bit_kullm_5.8b_version2/outputs'

In [ ]:
# lora_config = LoraConfig.from_pretrained(output_dir)
# model_get_peft = get_peft_model(model, lora_config)

#  3.Test

In [ ]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
def gen(question,context):
    gened = model.generate(
        **tokenizer(
            f"### 질문:{question}? 맥락을 참조해서 답변해고, 알 수 없는 글자는 적절히 보정해줘. 맥락에서 답변을 구할 수 없으면 절대 답변하지마. \n\n### 맥락:{context} \n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    return tokenizer.decode(gened[0])

In [ ]:
# 질문
question = '채권이 뭐야?'

# query it
docs = DB.similarity_search(question)
similar = docs[0].page_content

# Extract the content after '### 맥락:'
start_idx = similar.find("### 맥락:") + len("### 맥락:")
end_idx = similar.find("### 답변:")

context = "## 맥락:" + similar[start_idx:end_idx].strip() + " ### 답변:" + similar[end_idx+len("### 답변:"):].strip()

# 답변 생성
answer = gen(question, context)

def extract_first_answer(text):
    # "### 답변:" 뒤의 내용을 찾음
    start_index = text.find("### 답변:")
    if start_index == -1:
        return None
    start_index += len("### 답변:")

    # 답변 내용 중 "다."로 끝나는 첫번째 문장을 찾음
    end_index = text.find("다.", start_index) + len("다.")

    return text[start_index:end_index].strip()

result = extract_first_answer(answer)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
result

'채권은 투자자로부터 상대적으로 장기적인 자금을 조달하고 일정 기간 동안 정기적으로 이자를 지급하고 만기 시 원금을 상환하는 조건을 명시한 증권입니다.'